# **AVOTOX**

# Data Collection and Preprocessing

In [8]:
!pip install scikit-learn==1.2.2
!pip install pandas==2.0.3
!pip install contractions
!pip install xgboost
!pip install imbalanced-learn


In [19]:
import pandas as pd
import contractions
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.multioutput import MultiOutputClassifier
import joblib

**stopwords** are words that are filtered out of natural language data do to be considred unimportant (during or after processing)

In [20]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/heidyhernandez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/heidyhernandez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
vectorizer = joblib.load('vectorizer.pkl')
model = joblib.load('multi_output_model.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'vectorizer.pkl'

In [ ]:
def preprocess_text(text):
  text = text.lower()
  # Expand contractions
  text = contractions.fix(text)
  # Remove HTML tags
  text = re.sub(r'<.*?>', '', text)
  # Remove URLs
  text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
  # Remove email addresses
  text = re.sub(r'\S+@\S+', '', text)
  # Remove hashtags
  text = re.sub(r'#\w+', '', text)
  # Remove mentions
  text = re.sub(r'@\w+', '', text)
  # Remove special characters except punctuation
  text = re.sub(r'[^a-zA-Z\s.,]', '', text)
  # Remove extra whitespace
  text = re.sub(r'\s+', ' ', text).strip()
  # Tokenize text
  words = text.split()
  # Remove stop words
  stop_words = set(stopwords.words('english'))
  words = [word for word in words if word not in stop_words]
  # Lemmatize words
  lemmatizer = WordNetLemmatizer()
  words = [lemmatizer.lemmatize(word) for word in words]
  # Join words to a single string
  text = ' '.join(words)
  # print(text)
  return text

In [12]:
# Test the preprocessing with a sample comment
comment = "FUCK YOU!"
preprocessed_comment = preprocess_text(comment)

# Vectorize the preprocessed comment and make a prediction
comment_vector = vectorizer.transform([preprocessed_comment])
prediction = model.predict(comment_vector)
print("Prediction result:", prediction)

Prediction result: [[1 0 1 0 1 0]]


Download the CSV files via this link: [Kaggle Dataset](https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/data)

Then use Google Colab File Explorer:


*   On the left sidebar, click the Files tab.
*   Click the Upload button and select the files you want to upload.
*   Once uploaded, the files will appear in the file explorer.






In [20]:
# Load the files into pandas DataFrames
train_data = pd.read_csv('csv-files/train.csv')
test_data = pd.read_csv('csv-files/test.csv')
test_labels = pd.read_csv('csv-files/test_labels.csv')

# Dislay the first few rows of the datasets
print("train_data")
print(train_data.head())
print("test_data")
print(test_data.head())
print("test_labels")
print(test_labels.head())

train_data
                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  
test_data
                 id                                       comment_text
0  00001cee341fdb12  Yo bitch Ja Rule is more succesful then you'll...
1 

In [16]:
print(train_data.sample(50))

                      id                                       comment_text  \
139074  e83c7bb704364436  Article needs improvement \n\nArticle is just ...   
73305   c421c5c8f1f595e7  I Have a Hacker with me. \n\nIf you continue t...   
60861   a2f2d8a7c37c4d83                           This is a sock of banned   
123192  930f827d1df03973  Bilateral relationships need searching \n\nThe...   
70314   bc1eef29e9c6c4a2  "\n\n Personally speaking (of course) I think ...   
88892   edd28981dd206902  "\n\n th \n\n  th thanks for deleting my fucki...   
112321  58dcd559878c0190  Hello\n\nI just got a final warning??? from a ...   
20361   35c28ee12bdfc238                      what the fuck i do not get it   
89237   eeb6e7273ff2d958  There are two sources already provided for the...   
20240   3573bae301c74350  Please read up on Wikipedia's core policies be...   
9750    19d2c085364d1ca4  Might want to point out experts 99.9% of biolo...   
8754    173f912659fd0ca1  There are many biographies

In [17]:
print(test_data[:4])

                 id                                       comment_text
0  00001cee341fdb12  Yo bitch Ja Rule is more succesful then you'll...
1  0000247867823ef7  == From RfC == \n\n The title is fine as it is...
2  00013b17ad220c46  " \n\n == Sources == \n\n * Zawe Ashton on Lap...
3  00017563c3f7919a  :If you have a look back at the source, the in...


In [14]:
print(train_data['toxic'].value_counts())
print(train_data['severe_toxic'].value_counts())
print(train_data['obscene'].value_counts())
print(train_data['threat'].value_counts())
print(train_data['insult'].value_counts())
print(train_data['identity_hate'].value_counts())


toxic
0    144277
1     15294
Name: count, dtype: int64
severe_toxic
0    157976
1      1595
Name: count, dtype: int64
obscene
0    151122
1      8449
Name: count, dtype: int64
threat
0    159093
1       478
Name: count, dtype: int64
insult
0    151694
1      7877
Name: count, dtype: int64
identity_hate
0    158166
1      1405
Name: count, dtype: int64


# Model Building and Training - FINISH THE CODE :D

**Training data set:** This is the largest subset used to train the model by adjusting its parameters. It helps the model learn the underlying patterns in the data.


**Validation data set:** We use this set to provide an unbiased evaluation of the model during the training phase.
Credit: [link text](https://kili-technology.com/training-data/training-validation-and-test-sets-how-to-split-machine-learning-data#:~:text=Training%20data%20set%3A%20This%20is,model%20during%20the%20training%20phase.)





[RandomForestClassifier scikit](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

[XGBoost
](https://xgboost.readthedocs.io/en/latest/python/python_api.html)

[More](https://stackoverflow.com/questions/45251126/deprecation-warning-on-xgboost-sklearn)

In [22]:
# Apply preprocessing
________['comment_text'] = train_data['comment_text'].apply(________)
________['comment_text'] = test_data['comment_text'].apply(________)

# Vectorize the text data
vectorizer = TfidfVectorizer(max_features=10000)
X_train = vectorizer.fit_transform(________)

y_train = train_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

# Split the data into training and validation sets
# HINT: think about the train variables 
X_train, X_val, y_train, y_val = train_test_split(________, ________, test_size=0.2, random_state=42)

# Define the RandomForestClassifier and XGBoostClassifier models
rf_model = ________(
    random_state=42, 
    n_estimators=100, 
    max_depth=10, 
    class_weight='balanced'  # Adjusts weights inversely proportional to class frequencies
)

xgb_model = ________(eval_metric='logloss', use_label_encoder=False)

# Combine the models in a VotingClassifier
ensemble_model = ________(estimators=[
    ('rf', ________),
    ('xgb', ________)
], voting='soft')

multi_output_model = MultiOutputClassifier(________)

# Fit the ensemble model
# HINT: think about the train variables 
multi_output_model.fit(________, ________)

# Evaluate the model on the validation set
y_val_pred = multi_output_model.predict(________)
print('Validation Set Results')
# HINT: sklearn.metrics.classification_report(y_true, y_pred, *, labels=None, target_names=None, 
# sample_weight=None, digits=2, output_dict=False, zero_division='warn')[source]
# SEARCH UP: classification_report for more info :D
print(classification_report(y_val, ________, target_names=y_train.columns, zero_division=________))

# Preprocess test data
X_test = vectorizer.transform(test_data['comment_text'])
y_test_pred = multi_output_model.predict(________)

# Filter out test labels with -1 
# HINT: What can get you the index attribute
valid_indices = test_labels[(test_labels[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']] != -1).all(axis=1)].________
y_test_true = test_labels.loc[valid_indices, ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values
# HINT: Replace with 0
y_test_true[y_test_true == ________] = ________
y_test_pred_filtered = y_test_pred[________]

# Evaluate the model on the test set
print('Test Set Results')
print(classification_report(y_test_true, ________, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], zero_division=________))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:51:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:52:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:52:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:52:56] WARNING: /Users/runner/work/xgboost/xgboost/src

Validation Set Results
               precision    recall  f1-score   support

        toxic       0.90      0.61      0.73      3056
 severe_toxic       0.49      0.30      0.37       321
      obscene       0.88      0.72      0.79      1715
       threat       0.56      0.27      0.36        74
       insult       0.77      0.58      0.66      1614
identity_hate       0.63      0.28      0.39       294

    micro avg       0.84      0.60      0.70      7074
    macro avg       0.71      0.46      0.55      7074
 weighted avg       0.83      0.60      0.69      7074
  samples avg       0.05      0.05      0.05      7074

Test Set Results
               precision    recall  f1-score   support

        toxic       0.64      0.70      0.67      6090
 severe_toxic       0.37      0.41      0.39       367
      obscene       0.67      0.68      0.68      3691
       threat       0.50      0.35      0.41       211
       insult       0.68      0.58      0.63      3427
identity_hate       0

**Toxic and Obscene** classes have relatively high precision and recall, indicating good performance.


**Severe Toxic, Threat, and Identity Hate** classes have lower precision and recall, indicating that the model struggles with these categories, likely due to class imbalance or insufficient distinctive features.